In [5]:
import cv2
from ultralytics import YOLO

import pyrealsense2 as rs
import numpy as np


# Load a model
model = YOLO("yolov8s-pose.pt")  # load a pretrained model

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Setting parameter values
t_lower = 250 # Lower Threshold
t_upper = 400  # Upper threshold


config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
#pipeline.start(config)
profile = pipeline.start(config)


depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
depth_min = -1 #meter
depth_max = 1.0 #meter

depth_intrin = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
color_intrin = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()

depth_to_color_extrin =profile.get_stream(rs.stream.depth).as_video_stream_profile().get_extrinsics_to( profile.get_stream(rs.stream.color))
color_to_depth_extrin =  profile.get_stream(rs.stream.color).as_video_stream_profile().get_extrinsics_to( profile.get_stream(rs.stream.depth))


def pointconvert(color_point):
        

            depth_point_ = rs.rs2_project_color_pixel_to_depth_pixel(
                depth_frame.get_data(), depth_scale,
                depth_min, depth_max,
                depth_intrin, color_intrin, depth_to_color_extrin, color_to_depth_extrin, color_point)
           # print(depth_point_) 
  
            if depth_point_[0]<0 or depth_point_[1]<0: 
                 print('camera is so near to object and depth cannot determine') 
            else:
                 depth = depth_frame.get_distance(round(depth_point_[0]) , round(depth_point_ [1]))
                 dx,dy, dz = rs.rs2_deproject_pixel_to_point(depth_intrin, color_point , depth) 
                 return dx,dy,dz
       


while True:
    # Read a frame from the video stream

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        global depth_frame 
        depth_frame= frames.get_depth_frame()
        global color_frame 
        color_frame= frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        img = np.asanyarray(color_frame.get_data())
        
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

    
        # Use the model to predict the poses in the frame
        results = model.predict(img,save=False,show_labels=False,conf = 0.5)
       # print('results',results)
        detection = results[0].plot()
        keypoints = (results[0].keypoints)
        print(keypoints)
        p=[]
        for i in range(keypoints.shape[1]):
            x = keypoints[0, i, 0].item()
            y = keypoints[0, i, 1].item()
            confidence = keypoints[0, i, 2]
            if i == 
         #   dx,dy,dz=pointconvert((x,y))
          #  p.append((dx,dy,dz))
            confidence = keypoints[0, i, 2]
            print(f"Keypoint {i+1}: x={x}, y={y}, confidence={confidence}")

       # print(p)    
            
        cv2.imshow('YOLOv8 Pose Detection', detection)

        # Exit the program if the user presses the 'q' key
        if cv2.waitKey(1) == ord('q'):
            break

# Release the video capture device and close the display window
cap.release()
cv2.destroyAllWindows()


   


0: 480x640 1 person, 88.5ms
Speed: 1.7ms preprocess, 88.5ms inference, 9.3ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 person, 79.9ms
Speed: 6.9ms preprocess, 79.9ms inference, 9.6ms postprocess per image at shape (1, 3, 640, 640)


tensor([[[2.5096e+02, 1.2572e+02, 9.2610e-01],
         [2.6501e+02, 1.0225e+02, 5.9378e-01],
         [2.2016e+02, 1.0388e+02, 9.3862e-01],
         [2.7500e+02, 1.1997e+02, 1.7850e-01],
         [1.5790e+02, 1.2580e+02, 9.6457e-01],
         [3.2295e+02, 2.7500e+02, 9.6588e-01],
         [6.3318e+01, 2.5783e+02, 9.4299e-01],
         [3.5265e+02, 4.4951e+02, 2.1533e-01],
         [1.3806e+01, 4.1342e+02, 2.0918e-01],
         [2.9691e+02, 4.7469e+02, 7.0003e-02],
         [1.1765e+02, 4.3172e+02, 1.3810e-01],
         [2.4919e+02, 4.8000e+02, 1.8901e-02],
         [9.2146e+01, 4.8000e+02, 1.8397e-02],
         [3.4836e+02, 3.8476e+02, 1.9228e-03],
         [1.8230e+02, 3.7322e+02, 2.1272e-03],
         [1.9613e+02, 4.2360e+02, 8.9726e-04],
         [2.4313e+02, 4.0357e+02, 8.6262e-04]]], device='cuda:0')
Keypoint 1: x=250.96157836914062, y=125.71820068359375, confidence=0.9260989427566528
Keypoint 2: x=265.00543212890625, y=102.25004577636719, confidence=0.5937772393226624
Keypoint 3


0: 480x640 1 person, 83.4ms
Speed: 6.7ms preprocess, 83.4ms inference, 14.1ms postprocess per image at shape (1, 3, 640, 640)


tensor([[[2.4802e+02, 1.3946e+02, 9.9690e-01],
         [2.7442e+02, 1.2018e+02, 9.9034e-01],
         [2.1847e+02, 1.1637e+02, 9.9717e-01],
         [3.0541e+02, 1.5105e+02, 7.4648e-01],
         [1.7565e+02, 1.4413e+02, 9.6510e-01],
         [3.3812e+02, 2.9326e+02, 9.5218e-01],
         [9.4452e+01, 2.5310e+02, 9.4716e-01],
         [3.3842e+02, 4.6957e+02, 2.6540e-01],
         [1.2900e+01, 4.1443e+02, 3.8871e-01],
         [3.0895e+02, 4.8000e+02, 1.4562e-01],
         [6.4963e+01, 4.8000e+02, 2.3116e-01],
         [2.2615e+02, 4.8000e+02, 2.7052e-02],
         [6.7269e+01, 4.8000e+02, 3.0569e-02],
         [2.9274e+02, 4.1196e+02, 3.3291e-03],
         [9.5723e+01, 3.7219e+02, 4.2623e-03],
         [2.4880e+02, 4.3390e+02, 2.5356e-03],
         [1.2348e+02, 4.3484e+02, 2.8094e-03]]], device='cuda:0')
Keypoint 1: x=248.017578125, y=139.46376037597656, confidence=0.9968980550765991
Keypoint 2: x=274.4151306152344, y=120.17852783203125, confidence=0.99033522605896
Keypoint 3: x=218.


0: 480x640 1 person, 80.1ms
Speed: 7.3ms preprocess, 80.1ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 640)


...


0: 480x640 1 person, 79.7ms
Speed: 2.9ms preprocess, 79.7ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)



tensor([[[3.7284e+02, 1.4295e+02, 9.9851e-01],
         [3.8921e+02, 1.1454e+02, 9.5999e-01],
         [3.4398e+02, 1.0343e+02, 9.9868e-01],
         [3.8096e+02, 1.2085e+02, 1.1215e-01],
         [2.4519e+02, 9.9180e+01, 9.9320e-01],
         [3.7068e+02, 2.5101e+02, 9.9705e-01],
         [9.9499e+01, 2.2908e+02, 9.5120e-01],
         [4.9861e+02, 4.2998e+02, 8.6602e-01],
         [8.2041e+00, 4.4989e+02, 1.3264e-01],
         [4.6223e+02, 1.8436e+02, 8.7191e-01],
         [1.2244e+02, 4.5061e+02, 2.3925e-01],
         [2.5849e+02, 4.8000e+02, 1.3417e-01],
         [9.2147e+01, 4.8000e+02, 2.8345e-02],
         [4.2417e+02, 3.9137e+02, 2.3941e-02],
         [2.6193e+02, 4.4338e+02, 5.0922e-03],
         [4.1401e+02, 4.5369e+02, 4.0368e-03],
         [3.1403e+02, 4.4008e+02, 9.0095e-04]]], device='cuda:0')
Keypoint 1: x=372.8360900878906, y=142.953125, confidence=0.9985138773918152
Keypoint 2: x=389.2140197753906, y=114.53704833984375, confidence=0.9599875211715698
Keypoint 3: x=343.98

0: 480x640 1 person, 79.7ms
Speed: 2.1ms preprocess, 79.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)



tensor([[[3.7252e+02, 1.4245e+02, 9.9817e-01],
         [3.8851e+02, 1.1676e+02, 9.5171e-01],
         [3.4498e+02, 1.0295e+02, 9.9834e-01],
         [3.7840e+02, 1.2712e+02, 1.0594e-01],
         [2.4516e+02, 1.0008e+02, 9.9237e-01],
         [3.6892e+02, 2.5401e+02, 9.9689e-01],
         [9.6740e+01, 2.3109e+02, 9.4604e-01],
         [4.9734e+02, 4.2983e+02, 8.7653e-01],
         [8.5730e+00, 4.5500e+02, 1.3473e-01],
         [4.5992e+02, 1.8460e+02, 8.7703e-01],
         [1.1547e+02, 4.5122e+02, 2.3806e-01],
         [2.5937e+02, 4.8000e+02, 1.2575e-01],
         [8.9615e+01, 4.8000e+02, 2.5449e-02],
         [4.2510e+02, 3.9038e+02, 2.0078e-02],
         [2.4926e+02, 4.4052e+02, 4.1107e-03],
         [4.1090e+02, 4.5376e+02, 3.1707e-03],
         [2.8944e+02, 4.4657e+02, 6.8187e-04]]], device='cuda:0')
Keypoint 1: x=372.5167236328125, y=142.4513397216797, confidence=0.9981745481491089
Keypoint 2: x=388.5107421875, y=116.76449584960938, confidence=0.9517054557800293
Keypoint 3: x=34

KeyboardInterrupt: 

In [ ]:
import cv2
from ultralytics import YOLO

import pyrealsense2 as rs
import numpy as np
import torch

# Load a model
model = YOLO("yolov8s-pose.pt")  # load a pretrained model

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Setting parameter values
t_lower = 250 # Lower Threshold
t_upper = 400  # Upper threshold


config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
#pipeline.start(config)
profile = pipeline.start(config)


depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
depth_min = -1 #meter
depth_max = 1.0 #meter

depth_intrin = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
color_intrin = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()

depth_to_color_extrin =profile.get_stream(rs.stream.depth).as_video_stream_profile().get_extrinsics_to( profile.get_stream(rs.stream.color))
color_to_depth_extrin =  profile.get_stream(rs.stream.color).as_video_stream_profile().get_extrinsics_to( profile.get_stream(rs.stream.depth))


def pointconvert(color_point):
        

        depth_point_ = rs.rs2_project_color_pixel_to_depth_pixel(
            depth_frame.get_data(), depth_scale,
            depth_min, depth_max,
            depth_intrin, color_intrin, depth_to_color_extrin, color_to_depth_extrin, color_point)
       # print(depth_point_) 

        if depth_point_[0]<0 or depth_point_[1]<0: 
             print('camera is so near to object and depth cannot determine') 
             dx,dy,dz = 0,0,0
        else:
             depth = depth_frame.get_distance(round(depth_point_[0]) , round(depth_point_ [1]))
             dx,dy, dz = rs.rs2_deproject_pixel_to_point(depth_intrin, color_point , depth) 


        return dx,dy,dz

try:

    while True:
        # Read a frame from the video stream

            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            global depth_frame 
            depth_frame= frames.get_depth_frame()
            global color_frame 
            color_frame= frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue

            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            img = np.asanyarray(color_frame.get_data())

            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)


            # Use the model to predict the poses in the frame
            results = model.predict(img,save=False,show_labels=False,conf = 0.5)
           # print('results',results)
            detection = results[0].plot()
            Keypoints = (results[0].keypoints)
           # print('2d points',Keypoints)
            keypoints_3d =[]


        #    try:
            for i in range(Keypoints.shape[2]):
                x,y = Keypoints[0,i,0],Keypoints[0,i,1]
                dx,dy,dz=pointconvert((x,y))
                keypoints_3d.append((dx,dy,dz))
                 # Extract X, Y, and Z coordinates from the 3D keypoints
                X = [keypoint[0] for keypoint in keypoints_3d]
                Y = [keypoint[1] for keypoint in keypoints_3d]
                Z = [keypoint[2] for keypoint in keypoints_3d]
                # Create a figure and a 3D axis
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            print(X)
            # Plot the 3D keypoints
            ax.scatter(X, Y, Z)
            connection =define_connections()
            # Plot the connections as lines
            for connection in connections:
                start = keypoints_3d[connection[0]]
                end = keypoints_3d[connection[1]]
                ax.plot([start[0], end[0]], [start[1], end[1]], [start[2], end[2]], 'k-')

            # Set labels for the three axes
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')

            # Show the 3D plot
            plt.show()

except:
    pass

   # print('3d points',points_3d)
    # left shoulder
#         try:
# #             if Keypoints[0, 5, 2] > 0.8 and Keypoints[0, 7, 2] > 0.8:
# #                 x1 = Keypoints[0, 5, 0]
# #                 y1 = Keypoints[0, 5, 1]
# #                 dx,dy,dz=pointconvert((x1,y1))
# #                 x2 = Keypoints[0,7,0]
# #                 y2 = Keypoints[0,7,1]
#                 points_3d =[]
#                 for i in range(17):
#                       dx,dy,dz=pointconvert((Keypoints[0,i,0]),Keypoints[0,i,1])
#                       points_3d = points_3d.append((dx,dy,dz))
                
#                 print('3d points',points_3d)
                
                
# #                 # Calculate the center point
# #                 center_x = int((x1 + x2) / 2)
# #                 center_y = int((y1 + y2) / 2)

# #                 # Calculate the slope
# #                 slope = (y2 - y1) / (x2 - x1)
# #                 print(slope,type(slope))
# #                 # Determine the length of the sides (e.g., based on the distance between the points)
# #                 side_length = int(((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5)

# #                 # Determine the angle of the center line based on the slope
# #                 angle = int(-1 * (180 / 3.14159) *np.arctan(slope.cpu().numpy()))

#         except:
#             pass
      #      cv2.rectangle(detection, (center_x - side_length // 2, center_y - side_length // 2),
       #                   (center_x + side_length // 2, center_y + side_length // 2), (0, 255, 0), 2)
      #      cv2.line(detection, (center_x - side_length // 2, center_y),
     #                (center_x + side_length // 2, center_y), (0, 0, 255), 2)
#
     #   dx,dy,dz=pointconvert((x,y))
      #  p.append((dx,dy,dz))
#         confidence = keypoints[0, i, 2]
#         print(f"Keypoint {i+1}: x={x}, y={y}, confidence={confidence}")

       # print(p)    
            


   

In [3]:
import cv2
from ultralytics import YOLO

import pyrealsense2 as rs
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Load a model
model = YOLO("yolov8s-pose.pt")  # load a pretrained model


pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)
align = rs.align(rs.stream.color)

frames = pipeline.wait_for_frames()
frames = pipeline.wait_for_frames()
frames = pipeline.wait_for_frames()


while True:
    # Read a frame from the video stream
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)

        profile = aligned_frames.get_profile()


        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()
        depth_intrinsics = depth_frame.profile.as_video_stream_profile().get_intrinsics()
     #   print(depth_intrinsics)
        spatial = rs.spatial_filter()
        #spatial.set_option(rs.option.filter_magnitude, 5)
        #spatial.set_option(rs.option.filter_smooth_alpha, 1)
        #spatial.set_option(rs.option.filter_smooth_delta, 50)
        filtered_depth = spatial.process(depth_frame)

        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(filtered_depth.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)


        # Use the model to predict the poses in the frame
        results = model.predict(color_image,save=False,show_labels=False,conf = 0.7)
       # print('results',results)
        detection = results[0].plot()
        Keypoints = (results[0].keypoints).cpu().numpy()
     #   print('2d points',Keypoints)
        keypoint_3d =[]


        for i in range(12):
            center =[round(Keypoints[0,i,0]),round(Keypoints[0,i,1])]
            if center[1]==480:
                center[1]=479
            if center[0]==640:
                center[0]=639
            depth_point_ =(round(center[0]) , round(center[1]))

            depth = depth_frame.get_distance(depth_point_[0],depth_point_ [1])

            dx,dy, dz = rs.rs2_deproject_pixel_to_point(depth_intrinsics, (float(depth_point_[0]),float(depth_point_[1])) , depth) 

            keypoint_3d.append([dx,dy,dz])
            cv2.circle(depth_colormap, center, 5, (0, 255, 255), 2)  # Red circle with thickness of 2 pixels

        print(keypoint_3d)
                
  
    #Display the image with the circle
       # pose_3d(keypoint_3d)
        cv2.imshow('YOLOv8 Pose Detection', detection)
        cv2.imshow('depth',depth_colormap)
            # Exit the program if the user presses the 'q' key
        if cv2.waitKey(1) == ord('q'):
             break

        
cv2.destroyAllWindows()



   


0: 480x640 1 person, 287.9ms
Speed: 2.0ms preprocess, 287.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)



[[-0.0390201210975647, -0.1026884987950325, 1.1759999990463257], [-0.02785641700029373, -0.15394148230552673, 1.1970000267028809], [-0.08679480105638504, -0.12377075850963593, 1.1920000314712524], [0.024305975064635277, -0.16916300356388092, 1.2360000610351562], [-0.12573345005512238, -0.09389566630125046, 1.2670000791549683], [0.19791913032531738, -0.008626827038824558, 1.3480000495910645], [-0.12743474543094635, 0.10165291279554367, 1.2040001153945923], [0.3609086871147156, 0.2151305377483368, 1.3840000629425049], [-0.10578539222478867, 0.3485787808895111, 1.1220000982284546], [0.24818940460681915, 0.3676959276199341, 1.2430000305175781], [-0.0569722056388855, 0.1493462771177292, 1.0750000476837158], [0.16215743124485016, 0.3871920108795166, 1.1430000066757202]]


0: 480x640 1 person, 287.9ms
Speed: 3.6ms preprocess, 287.9ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)



[[-0.01926611177623272, -0.11048740148544312, 1.156000018119812], [-0.007919993251562119, -0.15846826136112213, 1.1720000505447388], [-0.07148399949073792, -0.13081102073192596, 1.1670000553131104], [0.03640503063797951, -0.17254453897476196, 1.2310000658035278], [-0.12039923667907715, -0.09674564003944397, 1.277000069618225], [0.20682358741760254, -0.0175313763320446, 1.3480000495910645], [-0.12616461515426636, 0.09867124259471893, 1.1920000314712524], [0.36468759179115295, 0.21238358318805695, 1.38100004196167], [-0.10314547270536423, 0.3398798406124115, 1.0940001010894775], [0.2539186179637909, 0.36593055725097656, 1.2510000467300415], [-0.05073355883359909, 0.14321883022785187, 1.055999994277954], [0.17834357917308807, 0.38766413927078247, 1.1500000953674316]]


0: 480x640 1 person, 287.7ms
Speed: 2.3ms preprocess, 287.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)



...

[[0.16321933269500732, -0.18221978843212128, 0.48000001907348633], [0.17086422443389893, -0.1873828023672104, 0.47700002789497375], [0.15675297379493713, -0.19131115078926086, 0.4870000183582306], [0.1941397786140442, -0.1818830966949463, 0.46300002932548523], [0.14962662756443024, -0.1924896538257599, 0.49000000953674316], [0.198959082365036, -0.18070457875728607, 0.46000000834465027], [0.1151382178068161, -0.20152488350868225, 0.5130000114440918], [0.0747828409075737, -0.008546732366085052, 0.32600000500679016], [0.018752124160528183, 0.004965289030224085, 0.4220000207424164], [0.08344709873199463, 0.01433434709906578, 0.320000022649765], [0.11931905895471573, -0.027057865634560585, 0.24500000476837158], [0.22402045130729675, 0.1373189091682434, 0.44200003147125244]]


0: 480x640 1 person, 287.1ms
Speed: 2.6ms preprocess, 287.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)



[[0.0, -0.0, 0.0], [0.15237656235694885, -0.1276717185974121, 0.32500001788139343], [0.0, -0.0, 0.0], [0.1535128802061081, -0.1272788792848587, 0.3240000009536743], [0.1669764220714569, -0.16695532202720642, 0.42500001192092896], [0.0, -0.0, 0.0], [0.1497780978679657, -0.16813383996486664, 0.4280000329017639], [0.11603698134422302, -0.023768465965986252, 0.3800000250339508], [0.11055214703083038, -0.014233484864234924, 0.39400002360343933], [0.14810225367546082, -0.09364219754934311, 0.31700000166893005], [0.15140239894390106, -0.11261969804763794, 0.31300002336502075], [0.21824368834495544, 0.13635674118995667, 0.42100003361701965]]


0: 480x640 (no detections), 286.5ms
Speed: 2.4ms preprocess, 286.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [3]:
def pointconvert(depth_intrin,color_point):
        
    depth_point_ =(round(color_point[0]) , round(color_point_[1]))
    depth = depth_frame.get_distance(depth_point_[0],depth_point_ [1])
    dx,dy, dz = rs.rs2_deproject_pixel_to_point(depth_intrin, (float(depth_point_[0]),float(depth_point_[1])) , depth) 
    return dx,dy, dz



In [4]:
def define_connections():
    connections = [
        (0, 1),   # Nose - Left eye
        (0, 2),   # Nose - Right eye
        (1, 3),   # Left eye - Left ear
        (2, 4),   # Right eye - Right ear
        (5, 6),   # Left shoulder - Right shoulder
        (5, 7),   # Left shoulder - Left elbow
        (6, 8),   # Right shoulder - Right elbow
        (7, 9),   # Left elbow - Left wrist
        (8, 10),  # Right elbow - Right wrist
        (5, 11),  # Left shoulder - Left hip
        (6, 12),  # Right shoulder - Right hip
        (11, 13), # Left hip - Left knee
        (12, 14), # Right hip - Right knee
        (13, 15), # Left knee - Left ankle
        (14, 16)  # Right knee - Right ankle
    ]
    return connections

In [2]:

def pose_3d(keypoints_3d):
    # Define the connections between keypoints to form the skeleton
#     connections = define_connections()

    # Extract X, Y, and Z coordinates from the 3D keypoints
    X = [keypoint[0] for keypoint in keypoints_3d]
    Y = [keypoint[1] for keypoint in keypoints_3d]
    Z = [keypoint[2] for keypoint in keypoints_3d]

    # Create a figure and a 3D axis
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot the 3D keypoints
    ax.scatter(X, Y, Z)

#     # Plot the connections as lines
#     for connection in connections:
#         start = keypoints_3d[connection[0]]
#         end = keypoints_3d[connection[1]]
#         ax.plot([start[0], end[0]], [start[1], end[1]], [start[2], end[2]], 'k-')

    # Set labels for the three axes
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Show the 3D plot
    plt.show()
